<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/250303_makeSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.0 MB/s eta 0:00:00


In [7]:
import os
import json
from openai import OpenAI
from google.colab import drive, userdata
from docx import Document

# ✅ Google Drive 마운트
drive.mount('/content/drive')

# ✅ OpenAI API 키 설정
api_key = userdata.get('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("API 키를 찾을 수 없습니다. userdata에 API 키를 저장했는지 확인하세요.")

os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

# ✅ 경로 설정
questions_dir = "/content/drive/MyDrive/LLM/Data/QnA/QuestionsSplitted"
sources_dir = "/content/drive/MyDrive/LLM/Data/voiceToText/clovaPaid"
summary_dir = "/content/drive/MyDrive/LLM/Data/Summary"
file_batch_dir = "/content/drive/MyDrive/LLM/Data/fileBatchIDs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# ✅ Summary 폴더 없으면 생성
os.makedirs(summary_dir, exist_ok=True)
os.makedirs(file_batch_dir, exist_ok=True)
# ✅ 폴더 내 파일 리스트 가져오기
json_files = [f for f in os.listdir(questions_dir) if f.endswith('.json')]

In [34]:
def upload_file(txt_file_path):
    attach_file = client.files.create(file=open(txt_file_path, "rb"), purpose="assistants")
    return attach_file.id

In [40]:
assistant = client.beta.assistants.create(
        instructions="첨부 txt와 json 파일을 이용해서 txt를 요약해줘",
        name="이사야 전문가",
        tools=[{"type": "file_search"}],
        # model="gpt-3.5-turbo-0125",
        model="gpt-4o-mini-2024-07-18"
    )

In [43]:
for json_file in json_files:
    if "사 1장 1-9 공부" not in json_file:
        continue

    base_name = os.path.splitext(json_file)[0]
    json_file_path = os.path.join(questions_dir, json_file)
    txt_file_path = os.path.join(sources_dir, base_name + ".txt")

    if not os.path.exists(txt_file_path):
        print(f"⚠️ 해당하는 TXT 파일이 없습니다: {txt_file_path}")
        continue

    # Create a vector store caled "Financial Statements"
    vector_store = client.beta.vector_stores.create(name="Isaiah Expert")

    # Ready the files for upload to OpenAI
    file_paths = [txt_file_path]
    file_streams = [open(path, "rb") for path in file_paths]

    # Use the upload and poll SDK helper to upload the files, add them to the vector store,
    # and poll the status of the file batch for completion.
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams
    )

    # You can print the status and the file counts of the batch to see the result of this operation.
    print(file_batch.status)
    print(file_batch.file_counts)

    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )

    # Upload the user provided file to OpenAI
    message_file = client.files.create(
        file=open(json_file_path, "rb"), purpose="assistants"
    )

    # Create a thread and attach the file to the message
    thread = client.beta.threads.create(
    messages=[
        {
        "role": "user",
        "content": "첨부 파일을 이용해서 txt를 요약해줘",
        # Attach the new file to the message.
        "attachments": [
            { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
        ],
        }
    ]
    )

    # The thread now has a vector store with that file in its tool resources.
    print(thread.tool_resources.file_search)

    # Use the create and poll SDK helper to create a run and poll the status of
    # the run until it's in a terminal state.

    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id=assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

    message_content = messages[0].content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
        message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
        if file_citation := getattr(annotation, "file_citation", None):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
ToolResourcesFileSearch(vector_store_ids=['vs_67c5bc4d407481919a5c606eed8f1cd4'])
첨부된 파일의 내용은 이사야서에 대한 논의로, 유다와 예루살렘의 영적 상태를 분석하며 하나님과의 관계에 대한 경고와 회복의 가능성을 다루고 있습니다. 다음은 주요 내용을 요약한 것입니다.

1. **하나님의 부르심과 경고**: 이사야는 하나님께서 유다와 예루살렘에 대해 경고하는 비전을 받습니다. 하나님은 자신의 백성이 자신을 거스르는 상황에 슬퍼하며 그들의 영적 상태를 진단합니다[0].

2. **부패한 사회**: 유다 백성은 하나님과의 언약을 잊고 범죄하며 부패한 생활을 하고 있습니다. 이사야는 그들의 상태를 '병든 머리와 피곤한 마음'으로 표현하면서 그들이 처한 영적 황폐함을 강조합니다[1].

3. **회복의 가능성**: 하나님은 상황이 완전히 무너져 가기 전에 돌아오기를 바라는 마음으로 이사야에게 메시지를 전달합니다. 초라한 남은 자들 속에서 여전히 소망이 존재함을 알리며, 그들을 통해 다시 회복할 수 있는 기회를 제공합니다[2][3].

4. **하나님과의 관계의 회복**: 이사야는 백성이 하나님을 다시 찾고 그와의 관계를 회복해야 함을 강조합니다. 상황이 어렵고 불확실할 때일수록 하나님의 인도하심을 믿고 의지해야 한다고 경고합니다[4].

5. **현재의 교훈**: 이 메시지는 오늘날 신자들에게도 적용될 수 있으며, 자신들의 상태를 인식하고 하나님께 돌아가야 할 필요성을 일깨워줍니다. 그들은 매일의 삶에서 하나님의 말씀에 귀 기울여야 하며, 세상의 소음에 휩쓸리지 않도록 주의해야 합니다[5][6].

이 사역은 이사야의 개인적 경험과 그가 이스라엘 백성에게 전달하고자 했던 하나님 메시지의 중요성을 강조합니다. 이를 통해 신앙인들은 단지 과거의 이야기가 아니라 현재